![piggy_bank](bank-logo.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

## 1. Reading in and splitting the data

In [2]:
df = pd.read_csv('bank_marketing.csv')

In [3]:
df

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,41185,56,retired,married,university.degree,no,yes,no,cellular,nov,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,41186,44,technician,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [4]:
client_df = df[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]

In [5]:
client_df

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no
41184,41184,46,blue-collar,married,professional.course,no,no,no
41185,41185,56,retired,married,university.degree,no,yes,no
41186,41186,44,technician,married,professional.course,no,no,no


In [6]:
campaign_df = df[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]

In [7]:
campaign_df

,client_id,campaign,month,day,duration,pdays,previous,poutcome,y
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,nonexistent,yes
41184,41184,1,nov,6,383,999,0,nonexistent,no
41185,41185,2,nov,24,189,999,0,nonexistent,no
41186,41186,1,nov,17,442,999,0,nonexistent,yes


In [8]:
economics_df = df[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

In [9]:
economics_df

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


## 2. Cleaning the data

In [10]:
client_df.rename(columns={"client_id": "id"}, inplace=True, errors='raise')

/tmp/ipykernel_86678/1827829544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df.rename(columns={"client_id": "id"}, inplace=True, errors='raise')


In [11]:
client_df

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no
41184,41184,46,blue-collar,married,professional.course,no,no,no
41185,41185,56,retired,married,university.degree,no,yes,no
41186,41186,44,technician,married,professional.course,no,no,no


In [12]:
campaign_df.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}, inplace=True, errors='raise')

/tmp/ipykernel_86678/1673604857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}, inplace=True, errors='raise')


In [13]:
campaign_df

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,nonexistent,yes
41184,41184,1,nov,6,383,999,0,nonexistent,no
41185,41185,2,nov,24,189,999,0,nonexistent,no
41186,41186,1,nov,17,442,999,0,nonexistent,yes


In [14]:
economics_df.rename(columns={'euribor3m':'euribor_three_months','nr_employed':'number_employed'}, inplace=True, errors='raise')

/tmp/ipykernel_86678/3555403141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  economics_df.rename(columns={'euribor3m':'euribor_three_months','nr_employed':'number_employed'}, inplace=True, errors='raise')


In [15]:
economics_df

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


In [16]:
client_df['education']=client_df['education'].str.replace('.','-')

/tmp/ipykernel_86678/3995786745.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client_df['education']=client_df['education'].str.replace('.','-')
/tmp/ipykernel_86678/3995786745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df['education']=client_df['education'].str.replace('.','-')


In [17]:
client_df['education'] = client_df['education'].replace('unknown', np.nan)



/tmp/ipykernel_86678/598279158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df['education'] = client_df['education'].replace('unknown', np.nan)


In [18]:
client_df

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic-4y,no,no,no
1,1,57,services,married,high-school,unknown,no,no
2,2,37,services,married,high-school,no,yes,no
3,3,40,admin.,married,basic-6y,no,no,no
4,4,56,services,married,high-school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional-course,no,yes,no
41184,41184,46,blue-collar,married,professional-course,no,no,no
41185,41185,56,retired,married,university-degree,no,yes,no
41186,41186,44,technician,married,professional-course,no,no,no


In [19]:
client_df["job"] = client_df["job"].str.replace(".", "")


/tmp/ipykernel_86678/1135528003.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client_df["job"] = client_df["job"].str.replace(".", "")
/tmp/ipykernel_86678/1135528003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_df["job"] = client_df["job"].str.replace(".", "")


In [20]:
client_df

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic-4y,no,no,no
1,1,57,services,married,high-school,unknown,no,no
2,2,37,services,married,high-school,no,yes,no
3,3,40,admin,married,basic-6y,no,no,no
4,4,56,services,married,high-school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional-course,no,yes,no
41184,41184,46,blue-collar,married,professional-course,no,no,no
41185,41185,56,retired,married,university-degree,no,yes,no
41186,41186,44,technician,married,professional-course,no,no,no


In [21]:
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].replace(('yes',1),('no',0))

/tmp/ipykernel_86678/4244028506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].replace(('yes',1),('no',0))


In [22]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace('nonexistent', np.nan)

/tmp/ipykernel_86678/2973286752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace('nonexistent', np.nan)


In [23]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace(('success',1),('failure',0))

/tmp/ipykernel_86678/288166474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace(('success',1),('failure',0))


In [24]:
campaign_df

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,NaN,no
1,1,1,may,19,149,999,0,NaN,no
2,2,1,may,23,226,999,0,NaN,no
3,3,1,may,27,151,999,0,NaN,no
4,4,1,may,3,307,999,0,NaN,no
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,NaN,no
41184,41184,1,nov,6,383,999,0,NaN,no
41185,41185,2,nov,24,189,999,0,NaN,no
41186,41186,1,nov,17,442,999,0,NaN,no


In [25]:
campaign_df['campaign_id']=1

/tmp/ipykernel_86678/1744431624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['campaign_id']=1


In [26]:
campaign_df['month'] = campaign_df['month'].str.capitalize()

/tmp/ipykernel_86678/2738719478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['month'] = campaign_df['month'].str.capitalize()


In [27]:
campaign_df['year'] = '2022-'

/tmp/ipykernel_86678/3407411504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['year'] = '2022-'


In [28]:
campaign_df['day'] = campaign_df['day'].astype(str)

/tmp/ipykernel_86678/3390391990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['day'] = campaign_df['day'].astype(str)


In [29]:
 campaign_df['last_contact_date']= campaign_df['year'] + campaign_df['month']+ '-' + campaign_df['day']

/tmp/ipykernel_86678/3014526940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df['last_contact_date']= campaign_df['year'] + campaign_df['month']+ '-' + campaign_df['day']


In [30]:
# campaign_df.drop(['month'])
campaign_df.drop(['year', 'month', 'day'], axis = 1, inplace = True) 

/tmp/ipykernel_86678/3197533707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df.drop(['year', 'month', 'day'], axis = 1, inplace = True)


In [31]:
campaign_df["last_contact_date"] = pd.to_datetime(campaign_df["last_contact_date"], 
                                               format="%Y-%b-%d")

/tmp/ipykernel_86678/28890814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign_df["last_contact_date"] = pd.to_datetime(campaign_df["last_contact_date"],


In [32]:
campaign_df

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,no,1,2022-05-13
1,1,1,149,999,0,NaN,no,1,2022-05-19
2,2,1,226,999,0,NaN,no,1,2022-05-23
3,3,1,151,999,0,NaN,no,1,2022-05-27
4,4,1,307,999,0,NaN,no,1,2022-05-03
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,no,1,2022-11-30
41184,41184,1,383,999,0,NaN,no,1,2022-11-06
41185,41185,2,189,999,0,NaN,no,1,2022-11-24
41186,41186,1,442,999,0,NaN,no,1,2022-11-17


## 3. Saving the data

In [33]:
client_df.to_csv('client.csv', index=False)

In [34]:
campaign_df.to_csv('campaign.csv', index=False)

In [35]:
economics_df.to_csv('economics.csv', index=False)

## 4. Designing the database

In [36]:
client_table = """ CREATE TABLE client 
(
        id SERIAL PRIMARY KEY,
        age INTEGER,
        job TEXT,
        marital TEXT,
        education TEXT,
        credit_default BOOLEAN,
        housing BOOLEAN,
        loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [37]:
campaign_table = """ CREATE TABLE client 
(       client_id SERIAL PRIMARY KEY,
        number_contacts INTEGER,
        contact_duration INTEGER,
        pdays INTEGER,
        previous_campaign_contacts BOOLEAN,
        previous_outcome TEXT,
        campaign_outcome TEXT,
        campaign_id BOOLEAN,
        last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [38]:
economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""